In [2]:
from pprint import pprint as pp
import numpy as np
import cv2

In [3]:
class Line():
  def __init__(self, index, top, down) -> None:
    self.index = index
    self.down = down
    self.top = top

  def __getitem__(self, key):
    if key == 0: return self.index
    if key == 1: return self.top
    if key == 2: return self.down
    
  def __repr__(self):
    return ("{ index: " + str(self.index) + ", " +\
            "top: " + str(self.top) + ", "+ \
            "down: " + str(self.down) + "}")

In [73]:
img = cv2.imread(r"../images/input.jpg", cv2.IMREAD_GRAYSCALE)

# Convert BGR to HSV
# hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# define range of black color in HSV
lower_val = np.array([0,0,0])
upper_val = np.array([0,50,50])

# Threshold the HSV image to get only black colors
mask = cv2.inRange(img, 0, 100)

# invert mask to get black symbols on white background
mask_inv = cv2.bitwise_not(mask)

blank = np.ones(img.shape, np.uint8)*255
blank2 = np.ones(img.shape, np.uint8)*255
blank3 = np.ones(img.shape, np.uint8)*255

contours, hier = cv2.findContours(mask_inv,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)

for cnt in contours:
    if cv2.contourArea(cnt) > 800:
        approx = cv2.approxPolyDP(cnt,
                                    0.00005 * cv2.arcLength(cnt, True), True)
        if(len(approx) >= 5):
            cv2.drawContours(blank, [approx], 0, (0, 0, 0), 1)

graph = []

for j in range(121, mask_inv.shape[1]):
    higher = 0
    lower = 0

    cntBlank = 0
    lines = []

    for i in range(30, mask_inv.shape[0]):
        if mask_inv[i,j] == 255: 
            cntBlank += 1
            if lower == 0:
                continue
        
        if lower == 0: lower = i
        if mask_inv[i,j] != 255: 
            higher = i
            cntBlank = 0
        if cntBlank > 10:
            lines.append(Line(j, higher, lower))
            lower = 0
            higher = 0
    #if len(lines) > 0:
    for l in lines:
        graph.append(l)
    lines.clear()
    # cntLine = 0
    # if (higher + lower > 0):
    #     graph.append( (j, higher, lower) )


In [74]:
ecg = []

while (len(graph) != 0):
    temp = [graph.pop(0)]
    for i, k in enumerate(graph):
        
        if (temp[-1].index != k.index-1):
            # если эта точка не сосед предыдущей
            continue
        
        if (temp[-1].top > k.down or k.top > temp[-1].down):
            # если эта точка не сосед предыдущей
            continue
            
        # если эта точка все таки сосед предыдущей
        temp.append(graph.pop(i))  
        
    if (len(temp) != 0):
        ecg.append(temp)
        print(temp)
        print(">__________________________<")


[{ index: 121, top: 562, down: 561}]
>__________________________<
[{ index: 122, top: 561, down: 557}]
>__________________________<
[{ index: 123, top: 561, down: 542}]
>__________________________<
[{ index: 124, top: 548, down: 524}]
>__________________________<
[{ index: 125, top: 540, down: 518}]
>__________________________<
[{ index: 126, top: 520, down: 518}]
>__________________________<
[{ index: 127, top: 359, down: 359}]
>__________________________<
[{ index: 127, top: 519, down: 518}]
>__________________________<
[{ index: 128, top: 359, down: 347}]
>__________________________<
[{ index: 128, top: 519, down: 518}]
>__________________________<
[{ index: 129, top: 355, down: 327}]
>__________________________<
[{ index: 129, top: 519, down: 518}]
>__________________________<
[{ index: 130, top: 333, down: 314}]
>__________________________<
[{ index: 130, top: 351, down: 351}]
>__________________________<
[{ index: 130, top: 519, down: 518}]
>__________________________<
[{ index: 

[{ index: 222, top: 364, down: 363}]
>__________________________<
[{ index: 222, top: 565, down: 565}]
>__________________________<
[{ index: 223, top: 162, down: 161}]
>__________________________<
[{ index: 223, top: 364, down: 363}]
>__________________________<
[{ index: 223, top: 566, down: 565}]
>__________________________<
[{ index: 224, top: 162, down: 161}]
>__________________________<
[{ index: 224, top: 363, down: 362}]
>__________________________<
[{ index: 224, top: 566, down: 565}]
>__________________________<
[{ index: 225, top: 162, down: 161}]
>__________________________<
[{ index: 225, top: 363, down: 362}]
>__________________________<
[{ index: 225, top: 566, down: 565}]
>__________________________<
[{ index: 226, top: 162, down: 161}]
>__________________________<
[{ index: 226, top: 362, down: 361}]
>__________________________<
[{ index: 226, top: 566, down: 565}]
>__________________________<
[{ index: 227, top: 162, down: 161}]
>__________________________<
[{ index: 

[{ index: 306, top: 165, down: 163}]
>__________________________<
[{ index: 306, top: 367, down: 366}]
>__________________________<
[{ index: 306, top: 568, down: 549}]
>__________________________<
[{ index: 307, top: 165, down: 164}]
>__________________________<
[{ index: 307, top: 366, down: 364}]
>__________________________<
[{ index: 307, top: 568, down: 542}]
>__________________________<
[{ index: 308, top: 165, down: 164}]
>__________________________<
[{ index: 308, top: 364, down: 362}]
>__________________________<
[{ index: 308, top: 571, down: 542}]
>__________________________<
[{ index: 309, top: 165, down: 164}]
>__________________________<
[{ index: 309, top: 362, down: 360}]
>__________________________<
[{ index: 309, top: 572, down: 545}]
>__________________________<
[{ index: 310, top: 165, down: 164}]
>__________________________<
[{ index: 310, top: 360, down: 355}]
>__________________________<
[{ index: 310, top: 571, down: 566}]
>__________________________<
[{ index: 

[{ index: 401, top: 169, down: 168}]
>__________________________<
[{ index: 401, top: 372, down: 371}]
>__________________________<
[{ index: 402, top: 169, down: 168}]
>__________________________<
[{ index: 402, top: 372, down: 371}]
>__________________________<
[{ index: 402, top: 570, down: 568}]
>__________________________<
[{ index: 403, top: 169, down: 169}, { index: 404, top: 169, down: 169}]
>__________________________<
[{ index: 403, top: 372, down: 371}]
>__________________________<
[{ index: 403, top: 570, down: 568}]
>__________________________<
[{ index: 404, top: 372, down: 371}]
>__________________________<
[{ index: 404, top: 569, down: 568}]
>__________________________<
[{ index: 405, top: 170, down: 169}]
>__________________________<
[{ index: 405, top: 372, down: 371}]
>__________________________<
[{ index: 405, top: 569, down: 568}]
>__________________________<
[{ index: 406, top: 170, down: 169}]
>__________________________<
[{ index: 406, top: 372, down: 371}]
>__

[{ index: 502, top: 167, down: 166}]
>__________________________<
[{ index: 502, top: 373, down: 372}]
>__________________________<
[{ index: 502, top: 574, down: 570}]
>__________________________<
[{ index: 503, top: 167, down: 166}]
>__________________________<
[{ index: 503, top: 373, down: 372}]
>__________________________<
[{ index: 503, top: 571, down: 568}]
>__________________________<
[{ index: 504, top: 167, down: 166}]
>__________________________<
[{ index: 504, top: 373, down: 372}]
>__________________________<
[{ index: 504, top: 570, down: 567}]
>__________________________<
[{ index: 505, top: 168, down: 166}]
>__________________________<
[{ index: 505, top: 373, down: 373}]
>__________________________<
[{ index: 505, top: 569, down: 565}]
>__________________________<
[{ index: 506, top: 168, down: 166}]
>__________________________<
[{ index: 506, top: 374, down: 372}]
>__________________________<
[{ index: 506, top: 568, down: 557}]
>__________________________<
[{ index: 

[{ index: 581, top: 374, down: 373}]
>__________________________<
[{ index: 582, top: 88, down: 77}]
>__________________________<
[{ index: 582, top: 172, down: 171}]
>__________________________<
[{ index: 582, top: 374, down: 372}]
>__________________________<
[{ index: 583, top: 88, down: 77}]
>__________________________<
[{ index: 583, top: 172, down: 170}]
>__________________________<
[{ index: 583, top: 374, down: 373}]
>__________________________<
[{ index: 583, top: 576, down: 575}]
>__________________________<
[{ index: 584, top: 95, down: 78}]
>__________________________<
[{ index: 584, top: 172, down: 171}]
>__________________________<
[{ index: 584, top: 374, down: 373}]
>__________________________<
[{ index: 584, top: 576, down: 575}]
>__________________________<
[{ index: 585, top: 95, down: 78}]
>__________________________<
[{ index: 585, top: 172, down: 171}]
>__________________________<
[{ index: 585, top: 374, down: 373}]
>__________________________<
[{ index: 585, top

[{ index: 696, top: 354, down: 353}]
>__________________________<
[{ index: 696, top: 575, down: 559}]
>__________________________<
[{ index: 697, top: 176, down: 175}]
>__________________________<
[{ index: 697, top: 354, down: 353}]
>__________________________<
[{ index: 697, top: 572, down: 551}]
>__________________________<
[{ index: 698, top: 176, down: 175}]
>__________________________<
[{ index: 698, top: 354, down: 353}]
>__________________________<
[{ index: 698, top: 642, down: 543}]
>__________________________<
[{ index: 699, top: 176, down: 175}]
>__________________________<
[{ index: 699, top: 354, down: 353}]
>__________________________<
[{ index: 699, top: 642, down: 540}]
>__________________________<
[{ index: 700, top: 177, down: 167}]
>__________________________<
[{ index: 700, top: 354, down: 353}]
>__________________________<
[{ index: 700, top: 641, down: 540}]
>__________________________<
[{ index: 701, top: 177, down: 158}]
>__________________________<
[{ index: 

[{ index: 822, top: 597, down: 591}]
>__________________________<
[{ index: 823, top: 92, down: 88}]
>__________________________<
[{ index: 823, top: 181, down: 180}]
>__________________________<
[{ index: 823, top: 314, down: 308}]
>__________________________<
[{ index: 823, top: 369, down: 363}]
>__________________________<
[{ index: 823, top: 503, down: 503}]
>__________________________<
[{ index: 823, top: 599, down: 596}]
>__________________________<
[{ index: 824, top: 94, down: 89}]
>__________________________<
[{ index: 824, top: 183, down: 180}]
>__________________________<
[{ index: 824, top: 312, down: 307}]
>__________________________<
[{ index: 824, top: 370, down: 365}]
>__________________________<
[{ index: 824, top: 599, down: 596}]
>__________________________<
[{ index: 825, top: 97, down: 90}]
>__________________________<
[{ index: 825, top: 184, down: 182}]
>__________________________<
[{ index: 825, top: 309, down: 304}]
>__________________________<
[{ index: 825, t

>__________________________<
[{ index: 957, top: 390, down: 357}]
>__________________________<
[{ index: 957, top: 588, down: 586}]
>__________________________<
[{ index: 958, top: 186, down: 184}]
>__________________________<
[{ index: 958, top: 390, down: 345}]
>__________________________<
[{ index: 958, top: 587, down: 586}]
>__________________________<
[{ index: 959, top: 186, down: 184}]
>__________________________<
[{ index: 959, top: 363, down: 345}]
>__________________________<
[{ index: 959, top: 390, down: 378}]
>__________________________<
[{ index: 959, top: 587, down: 586}]
>__________________________<
[{ index: 960, top: 186, down: 184}]
>__________________________<
[{ index: 960, top: 390, down: 389}]
>__________________________<
[{ index: 960, top: 587, down: 586}]
>__________________________<
[{ index: 961, top: 186, down: 184}]
>__________________________<
[{ index: 961, top: 390, down: 389}]
>__________________________<
[{ index: 961, top: 587, down: 586}]
>_________

[{ index: 1121, top: 196, down: 194}]
>__________________________<
[{ index: 1121, top: 390, down: 388}]
>__________________________<
[{ index: 1121, top: 592, down: 590}]
>__________________________<
[{ index: 1122, top: 195, down: 193}]
>__________________________<
[{ index: 1122, top: 391, down: 388}]
>__________________________<
[{ index: 1122, top: 592, down: 591}]
>__________________________<
[{ index: 1123, top: 194, down: 192}]
>__________________________<
[{ index: 1123, top: 393, down: 390}]
>__________________________<
[{ index: 1123, top: 592, down: 591}]
>__________________________<
[{ index: 1124, top: 193, down: 190}]
>__________________________<
[{ index: 1124, top: 394, down: 391}]
>__________________________<
[{ index: 1124, top: 593, down: 591}]
>__________________________<
[{ index: 1125, top: 192, down: 189}]
>__________________________<
[{ index: 1125, top: 395, down: 392}]
>__________________________<
[{ index: 1125, top: 593, down: 591}]
>_______________________

In [40]:
prev = graph[0]

lastAcc = prev[0]

prevPoint = 0
mainPoints = []
mainPoints.append(graph[0].top)
prevPoint = graph[0].down
status = False
print("Я начну с точки ", mainPoints[0])
clr = 50
img[mainPoints[0]][graph[0].index] = clr

for p in graph[1:]:
    
    #clr = 125*(p[0]%2)
    
    #if (p[0]%2 == 0):
    #    clr = 25
    #else:
    #    clr = 75
        
    if(status == False):
        if(mainPoints[-1] < p.top):
            if(prevPoint <= p.down):
                status = False
                mainPoints.append(p.top)
                prevPoint = p.down
            else:
                if((p.top - mainPoints[-1]) > (p.down - prevPoint)):
                    status = False
                    mainPoints.append(p.top)
                    prevPoint = p.down
                elif((p.top - mainPoints[-1]) < (p.down - prevPoint)):
                    status = True
                    mainPoints.append(p.down)
                    prevPoint = p.top
                else:
                    status = False
                    mainPoints.append(p.top)
                    prevPoint = p.down
        elif(mainPoints[-1] > p.top):
            if(prevPoint >= p.down):
                status = True
                mainPoints.append(p.down)
                prevPoint = p.top
            else:
                status = False
                mainPoints.append(p.top)
                prevPoint = p.down
        else:
            if(prevPoint <= p.down):
                status = False
                mainPoints.append(p.top)
                prevPoint = p.down
            else:
                status = True
                mainPoints.append(p.down)
                prevPoint = p.top
    else:
        if(mainPoints[-1] > p.down):
            if(prevPoint >= p.top):
                status = True
                mainPoints.append(p.down)
                prevPoint = p.top
            else:
                if((p.down - mainPoints[-1]) < (p.top - prevPoint)):
                    status = True
                    mainPoints.append(p.down)
                    prevPoint = p.top
                elif((p.down - mainPoints[-1]) > (p.top - prevPoint)):
                    status = False
                    mainPoints.append(p.top)
                    prevPoint = p.down
                else:
                    status = True
                    mainPoints.append(p.down)
                    prevPoint = p.top
        elif(mainPoints[-1] < p.down):
            status = False
            mainPoints.append(p.top)
            prevPoint = p.down
        else:
            if(p.top > prevPoint):
                status = False
                mainPoints.append(p.top)
                prevPoint = p.down
            else:
                status = True
                mainPoints.append(p.down)
                prevPoint = p.top
            

    #if ((p[1] <= prev[1] and p[1] >= prev[2]) or\
        #(p[2] <= prev[1] and p[2] >= prev[2])):
    
            #cv2.line(blank2, (p[0],p[2]), (p[0], p[1]), (clr,clr,clr))
            #prev = p
            #lastAcc = p[0]
    # else:
    #         print("")
    
    clr = 200
    
    cv2.line(img, (p[0],p[2]), (p[0], p[1]), clr)
    if(status == True):
        clr = 255
    else:
        clr = 0
    img[mainPoints[-1]][p.index] = clr
    cv2.line(blank3, (p[0],p[2]), (p[0], p[1]), clr)
    
cv2.imwrite(r'../images/mask_inv.png',mask_inv)
cv2.imwrite(r'../images/output2.png',img)
cv2.imwrite(r'../images/output.png',blank3)


IndexError: list index out of range